In [2]:
import sympy as sym
import numpy as np
from sympy import symbols, Eq
from sympy.abc import alpha, beta, theta, sigma, rho, phi, mu, nu

## Galí (2015) Sticky wages and sticky prices

In [32]:
y,pi_p, pi_w, w_ = symbols('y_t,\pi^p_t,\pi_t^w,\omega_{t-1}')
y1,pi_p1, pi_w1, w = symbols('E_ty_{t+1},E_t\pi^p_{t+1},E_t\pi_{t+1}^w,\omega_t')

phi_y, phi_p, phi_w=symbols('\phi_y, \phi_p, \phi_w')
lam_p, lam_w = symbols('\lambda_p, \lambda_w')
chi_p, chi_w = symbols('chi_p, chi_w')

v, Delta_w = symbols('v, \Delta\omega_t^n')

In [33]:
i = rho+phi_p*pi_p+phi_w*pi_w+phi_y*y+v

In [34]:
is_curve = Eq(y, y1-1/sigma*(i-pi_p1-rho))
real_wage = Eq(w, w_+pi_w-pi_p-Delta_w)
price_nkpc = Eq(pi_w, beta*pi_w1+chi_w*y-lam_w*w)
wage_nkpc = Eq(pi_p, beta*pi_p1+chi_p*y+lam_p*w)

In [35]:
var  = y1,pi_p1, pi_w1, w
var_ = y,pi_p, pi_w, w_
z = v, Delta_w

In [36]:
Gamma0,_ = sym.linear_eq_to_matrix([is_curve,real_wage,price_nkpc,wage_nkpc], var)
Gamma1,_ = sym.linear_eq_to_matrix(_,var_)
Psi, C = sym.linear_eq_to_matrix(_, z)
Eta = sym.Matrix([[0,1,0,0]]).T

In [37]:
calibration={'beta':0.99, 'phi':5, 'alpha':1/4, 'eps_p':9,
             '\phi_p':1.5,'\phi_w':0.125, '\phi_y':0.5/4,
             'eps_w':4.5, 'sigma':1,
              'theta_p':3/4,'theta_w':3/4}

In [38]:
class StickyWageNK():
    def __init__(self):
        self.calibration = None
    def calibrate(self, calibration: dict):
        theta_w = calibration['theta_w']
        theta_p = calibration['theta_p']
        beta = calibration['beta']
        phi = calibration['phi']
        eps_w = calibration['eps_w']
        eps_p = calibration['eps_w']
        alpha = calibration['alpha']
        sigma = calibration['sigma']
        calibration['\lambda_w']=(1-theta_w)*(1-beta*theta_w)/(theta_w*(1+eps_w*phi))
        calibration['\lambda_p']=(1-theta_p)*(1-beta*theta_p)/(theta_p)*(1-alpha)/(1-alpha+alpha*eps_p)
        calibration['chi_p'] = alpha*calibration['\lambda_p']/(1-alpha)
        calibration['chi_w'] = calibration['\lambda_w']*(sigma+phi/(1-alpha))
        self.calibration = calibration

In [39]:
economy=StickyWageNK()
economy.calibrate(calibration)

In [40]:
calibration = economy.calibration

In [41]:
Gamma0 = Gamma0.subs(calibration)
Gamma1 = Gamma1.subs(calibration)
C = C.subs(calibration)
Psi = Psi.subs(calibration)

In [ ]:
Gamma0_inv=Gamma0.inv()
Gamma1, C, Psi, Eta = Gamma0_inv@Gamma1, Gamma0_inv@C, Gamma0_inv@Psi, Gamma0_inv@Eta 
P, Lambda = Gamma1.diagonalize()
ind = list(np.argsort(np.abs(np.diag(np.abs(Lambda)))))
Lambda = sym.diag(*[Lambda[i,i] for i in ind])
P=P.permute_cols(ind)
P_inv = P.inv()
n_s = len([Lambda[i,i] for i in range(Lambda.rows) if np.abs(Lambda[i,i])<1])
n_u = Lambda.rows-n_s
Lambda_S=Lambda[:n_s,:n_s]
Lambda_U=Lambda[n_s:,n_s:]
Theta_1 = P[:,:n_s]*Lambda_S*P_inv[:n_s,:]
Theta_c = P[:,:n_s]*P_inv[:n_s,:]+P[:,n_s:]*(sym.eye(n_u)-Lambda_U)*P_inv[n_s:,:]
Phi = P[:,:n_s]*Eta*(P_inv[n_s:,:]*Eta).inv()
Theta_z = P[:,:n_s]*P_inv[:n_s,:]-P[:,:n_s]*Phi*P_inv[n_s:,:]

In [ ]:
def solve_system(Gamma0:sym.Matrix, Gamma1:sym.Matrix, C:sym.Matrix, Psi:sym.Matrix, Eta:sym.Matrix)->tuple:
    if np.abs(Gamma0.det())<1e-10:
        raise ValueError('Gamma0 is not invertible')
    Gamma0_inv=Gamma0.inv()
    Gamma_1, C, Psi, Eta = Gamma0_inv@Gamma_1, Gamma0_inv@C, Gamma0_inv@Psi, Gamma0_inv@Eta 
    P, Lambda = Gamma1.diagonalize()
    P_inv = P.inv()
    ind = list(np.argsort(np.abs(np.diag(np.abs(Lambda)))))
    Lambda = sym.diag(*[Lambda[i,i] for i in ind])
    P=P.permute_cols(ind)
    n_s = len([Lambda[i,i] for i in range(Lambda.rows) if np.abs(Lambda[i,i])<1])
    n_u = Lambda.rows-n_s
    Lambda_S=Lambda[:n_s,:n_s]
    Lambda_U=Lambda[n_s:,n_s:]
    Theta_1 = P*sym.diag(Lambda_S, sym.zeros((n_u,n_u)))*P_inv
    #Theta_c = P*sym.Matrix([[sym.eye(n_s)],(sym.eye(n_u)-Lambda_U).inv()])*P_inv
    #Theta_z = P*sym.Matrix([[sym.eye(Psi.rows), Psi],[0,0]])*P_inv
    Theta_c = None
    Theta_z = None
    return Theta_1, Theta_c, Theta_z